In [8]:
import os
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# from openai import AzureOpenAI  #? client.chat.completions.create
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

#? Model should also have context about document to answer it more appropriately
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [3]:
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['ENDPOINT_URL'] = os.getenv("ENDPOINT_URL")
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['AZURE_OPENAI_API_VERSION'] = os.getenv("AZURE_OPENAI_API_VERSION")

##### Basic llmAzure Usage

In [ ]:
'''

llm = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["ENDPOINT_URL"],
)
response = llm.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ],
)
print(json.dumps(response.to_dict(), indent=4))

llm = AzureChatOpenAI(
    model="gpt-4o",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["ENDPOINT_URL"],
)
llm.invoke("What is generative AI?")

prompt=ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answers based on the questions"),
    ("user","{input}")
    ])
prompt = ChatPromptTemplate.from_template("Question: {input}")
output_parser = StrOutputParser()
chain=prompt|llm|output_parser
response=chain.invoke({"input":"Can you tell me about Langsmith?"})

'''

##### GenAI App

##### MostBasic

In [38]:
docs=WebBaseLoader("https://docs.smith.langchain.com/observability/tutorials/observability").load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs=text_splitter.split_documents(docs)
texts = [doc.page_content for doc in docs]
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vectorstoredb = FAISS.from_documents(docs, embeddings)
# query="One Last part of the application we havent traced"
# result=vectorstoredb.similarity_search(query)
# result[0].page_content

##### DocumentChain-->Retrieval Chain 

In [41]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """)
llm = AzureChatOpenAI(
    model="gpt-4o",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["ENDPOINT_URL"],
)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain=create_retrieval_chain(vectorstoredb.as_retriever(),document_chain)
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']


'LangSmith provides tools to trace the whole LLM pipeline, not just individual calls to the LLM. To trace the entire pipeline, you need to import from `langsmith` and use the `@traceable` decorator on the overall function. For example, if you call the function `rag("where did harrison work")`, LangSmith will produce a trace of the entire pipeline including the OpenAI call as a child run.\n\nAdditionally, you will need to set environment variables to enable tracing, authenticate the API, and designate the project. Here are the relevant environment variables:\n\n- `LANGSMITH_TRACING=true`\n- `LANGSMITH_API_KEY=<your-api-key>`\n- `LANGSMITH_PROJECT=default`\n\nThese variables may be referenced as `LANGCHAIN_*` in other places, but `LANGSMITH_TRACING`, `LANGSMITH_API_KEY`, and `LANGSMITH_PROJECT` are considered the best practice. Ensure you\'re using the appropriate versions of the SDK to support the `LANGSMITH_PROJECT` flag.\n\nTracing your LLM calls is straightforward with LangSmith\'s O

##### LCEL

In [ ]:
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=os.environ['GROQ_API_KEY'])
generic_template="Translate the following into {language}:"
prompt=ChatPromptTemplate.from_messages([
    ("system",generic_template),("user","{text}")
])
human_query={"language":"French","text":"Hello"}
# prompt.invoke(human_query).to_messages()
# messages=[
#     SystemMessage(content="Translate the following from English to French"),
#     HumanMessage(content="Hello How are you?")
# ]
parser=StrOutputParser()
chain=prompt|model|parser
response=chain.invoke(human_query)
response

'Bonjour \n'